# Distance Plot
Notebook that generates a box-and-whisker plot for a given dataset.

### Configure Paths
In `MARIAM/data/` each folder is a *dataset*. Each dataset consists of a subfolder of trials for each controller type:
- `<MM-DD>_p_experiments/`: Data from position controller trials.
- `<MM-DD>_a_experiments/`: Data from admittance controller trials.
- `<MM-DD>_dp_experiments/`: Data from dynamic parameterization controller trials. 

Each trial folder consists of the following files:
- `monica_admittance_data.csv`: Admittance and manipulator data for Monica.
- `ross_admittance_data.csv`: Admittance and manipulator data for Ross.
- `trajectory_data.csv`: Locations and orientations of the payload, Monica, and Ross.

In [ ]:
# Dataset path
dataset_path = '../../data/08-30_dataset/'

# Controller type paths
dp_experiments_path = dataset_path + '08-30_dp_experiments/'
a_experiments_path = dataset_path + '08-30_a_experiments/'
p_experiments_path = dataset_path + '08-30_p_experiments/'

### Create Data Structure
All csv data across all trials will be organized in a singular data structure with the following format:
```python
data = {
    'p': {  # position controller
        'trial_01': {
            'trajectory': DataFrame,
            'monica_admittance': DataFrame, 
            'ross_admittance': DataFrame
        },
        'trial_02': { ... },
        ...
    },
    'a': { ... },  # admittance controller  
    'dp': { ... }  # dynamic parameterization controller
}
```

In [ ]:
import pandas as pd
import os
import re
from pathlib import Path

def load_experiment_data(dataset_path):
    """
    Load all robotic experiment data without preprocessing.
    
    Args:
        dataset_path: Path to the main dataset folder (e.g., '../../data/08-30_dataset/')
    
    Returns:
        dict: Nested dictionary with structure data[controller][trial_num][file_type]
    """
    
    data = {}
    date_prefix = Path(dataset_path).name.split('_')[0]
    
    # File naming conventions by controller type
    file_mappings = {
        'p': {'monica': 'monica_position_data.csv', 'ross': 'ross_position_data.csv'},
        'a': {'monica': 'monica_admittance_data.csv', 'ross': 'ross_admittance_data.csv'},
        'dp': {'monica': 'monica_admittance_data.csv', 'ross': 'ross_admittance_data.csv'}
    }
    
    for controller in ['p', 'a', 'dp']:
        controller_path = os.path.join(dataset_path, f"{date_prefix}_{controller}_experiments")
        
        if not os.path.exists(controller_path):
            print(f"Warning: Path {controller_path} does not exist.")
            continue
            
        print(f"Loading {controller} controller data...")
        data[controller] = {}
        
        # Process all trial folders
        trial_folders = [f for f in os.listdir(controller_path) 
                        if os.path.isdir(os.path.join(controller_path, f)) and 'trial' in f]
        
        for trial_folder in sorted(trial_folders):
            trial_match = re.search(r'trial_(\d+)', trial_folder)
            if not trial_match:
                continue
                
            trial_num = trial_match.group(1)
            trial_path = os.path.join(controller_path, trial_folder)
            
            # Load trajectory data
            trajectory_file = os.path.join(trial_path, 'trajectory_data.csv')
            if not os.path.exists(trajectory_file):
                print(f"  Warning: trajectory_data.csv not found in trial {trial_num}")
                continue
            
            try:
                data[controller][trial_num] = {
                    'trajectory': pd.read_csv(trajectory_file)
                }
                
                # Load monica and ross data files
                for robot, filename in file_mappings[controller].items():
                    file_path = os.path.join(trial_path, filename)
                    if os.path.exists(file_path):
                        try:
                            data[controller][trial_num][f"{robot}_{controller}"] = pd.read_csv(file_path)
                        except Exception as e:
                            print(f"    Error loading {filename} in trial {trial_num}: {e}")
                    else:
                        print(f"    Warning: {filename} not found in trial {trial_num}")
                
            except Exception as e:
                print(f"  Error loading trial {trial_num}: {e}")
    
    # Summary
    print("\n" + "="*40)
    print("LOADING SUMMARY")
    print("="*40)
    total_trials = 0
    for controller in data:
        trial_count = len(data[controller])
        total_trials += trial_count
        print(f"{controller.upper()}: {trial_count} trials")
    print(f"Total: {total_trials} trials")
    
    return data
    

data = load_experiment_data(dataset_path=dataset_path)

### Data Pre-Processing
Each The monica and ross CSV files for each trial will be cropped to the start and end times of the `trajectory_data.csv` file.

In [ ]:
def crop_experiment_data(data):
    """
    Crop all non-trajectory data to match trajectory time bounds.
    
    Args:
        data: Data structure from load_experiment_data()
    
    Returns:
        dict: Same structure with cropped data
    """
    
    cropped_data = {}
    
    for controller in data:
        print(f"Cropping {controller} controller data...")
        cropped_data[controller] = {}
        
        for trial_num in data[controller]:
            trial_data = data[controller][trial_num]
            
            # Get time bounds from trajectory data
            trajectory_df = trial_data['trajectory']
            start_time = trajectory_df['ros_time_ns'].min()
            end_time = trajectory_df['ros_time_ns'].max()
            
            # Initialize with trajectory data (no cropping needed)
            cropped_data[controller][trial_num] = {
                'trajectory': trajectory_df.copy()
            }
            
            # Crop all other data files
            for data_type, df in trial_data.items():
                if data_type == 'trajectory':
                    continue
                    
                if 'ros_time_ns' not in df.columns:
                    print(f"    Warning: ros_time_ns column not found in {data_type}, trial {trial_num}")
                    cropped_data[controller][trial_num][data_type] = df.copy()
                    continue
                
                # Crop to trajectory time range
                original_length = len(df)
                cropped_df = df[(df['ros_time_ns'] >= start_time) & (df['ros_time_ns'] <= end_time)].copy()
                
                # Check coverage and warn if incomplete
                df_start, df_end = df['ros_time_ns'].min(), df['ros_time_ns'].max()
                if df_start > start_time or df_end < end_time:
                    print(f"    Warning: {data_type} trial {trial_num} doesn't fully cover trajectory time range")
                
                # print(f"    {data_type} trial {trial_num}: {original_length} -> {len(cropped_df)} rows")
                cropped_data[controller][trial_num][data_type] = cropped_df
    
    print("\nCropping complete!")
    return cropped_data

processed_data = crop_experiment_data(data=data)

### Calculate Distance Travelled
The *distance traveled* is the distance the payload traveled before it was dropped. The time the payload is dropped is signaled by force value dropping below 0.5 for both robots. The total distance is calculated by summing the Euclidian distance from one time step to the next for each time step.

In [ ]:
import numpy as np

def calculate_distance_traveled(data):
    """
    Calculate distance traveled by payload for each trial before force drops below 0.5.
    
    Args:
        data: Data structure from load_experiment_data() or crop_experiment_data()
    
    Returns:
        dict: {controller: [distances]} where distances is list of distances for each trial
    """
    
    distance_results = {}
    
    for controller in data:
        print(f"Processing {controller} controller...")
        distances = []
        
        for trial_num in sorted(data[controller].keys()):
            trial_data = data[controller][trial_num]
            
            try:
                # Get data
                trajectory_df = trial_data['trajectory']
                monica_df = trial_data[f'monica_{controller}']
                ross_df = trial_data[f'ross_{controller}']
                
                # Find earliest time force < 0.5 for each robot
                monica_low_mask = monica_df['force'] < 0.01
                if monica_low_mask.any():
                    monica_low_force_time = monica_df[monica_low_mask]['ros_time_ns'].min()
                else:
                    monica_low_force_time = monica_df['ros_time_ns'].max()
                
                ross_low_mask = ross_df['force'] < 0.01
                if ross_low_mask.any():
                    ross_low_force_time = ross_df[ross_low_mask]['ros_time_ns'].min()
                else:
                    ross_low_force_time = ross_df['ros_time_ns'].max()
                
                # Take the larger timestamp
                low_force_time = max(monica_low_force_time, ross_low_force_time)
                
                # Filter trajectory data up to low_force_time
                filtered_trajectory = trajectory_df[trajectory_df['ros_time_ns'] <= low_force_time]
                
                # Calculate total distance
                payload_distances = np.sqrt(
                    np.diff(filtered_trajectory['act_payload_x'])**2 + 
                    np.diff(filtered_trajectory['act_payload_y'])**2
                )
                total_distance = min(np.sum(payload_distances), 2.20)
                
                distances.append(total_distance)
                # print(f"  Trial {trial_num}: {total_distance:.3f}")
                
            except Exception as e:
                print(f"  Error in trial {trial_num}: {e}")
        
        distance_results[controller] = distances
    
    return distance_results

# Try start to finish distance instead?
distance_results = calculate_distance_traveled(data=processed_data)

# Print all distances
print("Distances traveled for debugging:")
for controller, distances in distance_results.items():
    for distance in distances:
        print(f"  {controller} controller distance: {distance:.3f}")

### Success Rate
A hard-coded array of success rates. Trial succes is categorized in the following way:
- `perfect`: The payload reaches the goal with no hiccups.
- `faulty`: The payload reaches the goal but the payload was not held stably.
- `failed`: The payload is dropped before the goal.

In [ ]:
success_counts = {
    'p': { 'perfect': 0, 'faulty': 0, 'failed': 20 },
    'a': { 'perfect': 1, 'faulty': 4, 'failed': 15 },
    'dp': { 'perfect': 18, 'faulty': 0, 'failed': 7 }
}

# Calculate success rates using counts
success_rates = {
    controller: {
        'perfect': count['perfect'] / sum(count.values()) if sum(count.values()) > 0 else 0,
        'faulty': count['faulty'] / sum(count.values()) if sum(count.values()) > 0 else 0,
        'failed': count['failed'] / sum(count.values()) if sum(count.values()) > 0 else 0
    }
    for controller, count in success_counts.items()
}
success_rates

## Plots
This section uses data gathered previously to make the following plots:
- Distance Traveled / Success Rate, Box Plot
- Distance Traveled / Success Rate, Violin Plot

### Distance Traveled / Success Rate Box Plot

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

def create_distance_box_plot(distance_results, success_rates, fig_size=(18, 6), fontsize=16):
    """
    Create a notched box and whisker plot from distance_results with success rate bars.
   
    Args:
        distance_results: Dictionary from calculate_distance_traveled()
                         {controller: [distances]}
        success_rates: Dictionary with success rates
                      {controller: {'perfect': rate, 'faulty': rate, 'failed': rate}}
        fig_size: Figure size tuple
        fontsize: Font size for labels and text
    """
   
    # Convert distance_results to DataFrame for seaborn
    plot_data = []
    controller_labels = {
        'p': 'Position',
        'a': 'Admittance',
        'dp': 'Dynamic Param'
    }
   
    for controller, distances in distance_results.items():
        for distance in distances:
            plot_data.append({
                'experiment_type': controller_labels.get(controller, controller),
                'distance_traveled': distance
            })
   
    distance_df = pd.DataFrame(plot_data)
    controller_types = list(distance_df['experiment_type'].unique())
   
    # Find the maximum distance across all data
    max_distance = distance_df['distance_traveled'].max()
   
    # Create the plot with dual y-axes
    fig, ax1 = plt.subplots(figsize=fig_size)
    ax2 = ax1.twinx()
   
    sns.set_theme(
        style='white',
        font="Times New Roman"
    )
   
    # Create box plot on primary axis
    sns.boxplot(
        data=distance_df,
        x="experiment_type",
        y="distance_traveled",
        width=0.3,
        positions=[x - 0.15 for x in range(len(controller_types))],
        notch=True,
        showcaps=True,
        showmeans=True,
        flierprops={"marker": "x"},
        medianprops={
            "color": "orange",
            "linewidth": 1.5
        },
        boxprops={
            "facecolor": "white",
            "edgecolor": "black",
            "linewidth": 1.5,
            "alpha": 0.8
        },
        meanprops={
            "marker": "o",
            "markerfacecolor": "black",
            "markeredgecolor": "black",
            "markersize": 8
        },
        ax=ax1
    )
    
    # Add horizontal line at maximum distance
    ax1.axhline(y=max_distance, color='green', linestyle='--', linewidth=2, 
               label='Goal Distance', alpha=0.8)
    
    # Set up primary y-axis
    ax1.set_xlabel("", fontsize=fontsize, fontweight='semibold')
    ax1.set_ylabel("Distance (m)", fontsize=fontsize, fontweight='semibold')
    ax1.tick_params(axis='both', labelsize=fontsize)
    ax1.grid(True, alpha=0.3, linestyle='-', linewidth=0.5)
   
    # Create stacked bar plot on secondary axis
    x_positions = range(len(controller_types))
    
    # Prepare data for stacked bars
    perfect_rates = []
    faulty_rates = []
    
    for i, controller_type in enumerate(controller_types):
        # Find original controller key
        controller_key = None
        for key, label in controller_labels.items():
            if label == controller_type:
                controller_key = key
                break
        
        if controller_key and controller_key in success_rates:
            rates = success_rates[controller_key]
            perfect_rates.append(rates['perfect'] * 100)
            faulty_rates.append(rates['faulty'] * 100)
        else:
            perfect_rates.append(0)
            faulty_rates.append(0)
    
    # Offset bars to be next to boxes (slightly to the right)
    bar_width = 0.25
    bar_positions = [x + 0.15 for x in x_positions]
    
    # Create stacked bars with different colors and patterns
    bars1 = ax2.bar(bar_positions, perfect_rates, bar_width, 
                   label='Perfect', color='lightgreen', edgecolor='darkgreen', 
                   linewidth=1, alpha=0.8)
    bars2 = ax2.bar(bar_positions, faulty_rates, bar_width, 
                   bottom=perfect_rates, label='Faulty', color='orange', 
                   edgecolor='darkorange', linewidth=1, alpha=0.8, hatch='///')
    
    # Add text labels above bars
    for i, (perfect_rate, faulty_rate) in enumerate(zip(perfect_rates, faulty_rates)):
        x_pos = bar_positions[i]  # Use the same x position as the bars
        
        # Text above perfect bar (green section)
        if perfect_rate > 0:  # Only show if there's a perfect rate
            ax2.text(x_pos, perfect_rate + 0, f'{perfect_rate:.0f}%', 
                    ha='center', va='bottom', fontsize=fontsize-2, fontweight='bold')
        
        # Text above faulty bar (orange section) - shows combined perfect + faulty rate
        combined_rate = perfect_rate + faulty_rate
        if combined_rate > 0:  # Only show if there's a combined rate
            ax2.text(x_pos, combined_rate + 0, f'{combined_rate:.0f}%', 
                    ha='center', va='bottom', fontsize=fontsize-2, fontweight='bold')
    
    # Set up secondary y-axis (aligned with primary axis)
    ax2.set_ylabel("Success Rate (%)", fontsize=fontsize, fontweight='semibold')
    ax2.tick_params(axis='y', labelsize=fontsize)
    
    # Align the two y-axes so 0 matches 0% and max_distance matches 100%
    ax1_min, ax1_max = ax1.get_ylim()
    ax1.set_ylim(0, max_distance * 1.1)  # Add some padding
    
    # Set the secondary axis to scale proportionally
    # 100% should align with max_distance on the primary axis
    ax2.set_ylim(0, 100 * (max_distance * 1.1) / max_distance)  # Scale to match primary axis range
    
    # Add sample size annotations above each box
    for i, controller_type in enumerate(controller_types):
        subset = distance_df[distance_df['experiment_type'] == controller_type]
        n = len(subset)
       
        # Get the maximum y-value for this box (top of upper whisker or outliers)
        box_data = subset['distance_traveled'].values
        upper_whisker = box_data.max()
       
        # Place text slightly above the upper whisker
        y_min, y_max = ax1.get_ylim()
        ax1.text(i - 0.15, upper_whisker + (y_max - y_min) * 0.02, f'n={n}', 
                 ha='center', va='bottom', fontsize=fontsize-2, fontweight='normal')
    
    # Create legend in top left
    # Combine legends from both axes
    lines1, labels1 = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left', 
              fontsize=fontsize-2, framealpha=0.9)
       
    # Save as png and eps
    plt.savefig("./figures/distance_box_plot.png", dpi=300, bbox_inches='tight')
    plt.savefig("./figures/distance_box_plot.eps", bbox_inches='tight')
    
    # Print box plot statistics
    print("Box plot statistics:")
    print(f"Maximum distance (Goal): {max_distance:.2f}m")
    for controller_type in controller_types:
        subset = distance_df[distance_df['experiment_type'] == controller_type]
        print(f"{controller_type}:")
        print(f"  Mean: {subset['distance_traveled'].mean():.2f}")
        print(f"  Median: {subset['distance_traveled'].median():.2f}")
        print(f"  Std Dev: {subset['distance_traveled'].std():.2f}")
    
    # Print success rate statistics
    print("\nSuccess rate statistics:")
    for controller_type in controller_types:
        # Find original controller key
        controller_key = None
        for key, label in controller_labels.items():
            if label == controller_type:
                controller_key = key
                break
        
        if controller_key and controller_key in success_rates:
            rates = success_rates[controller_key]
            print(f"{controller_type}:")
            print(f"  Perfect: {rates['perfect']*100:.1f}%")
            print(f"  Faulty: {rates['faulty']*100:.1f}%")
            print(f"  Failed: {rates['failed']*100:.1f}%")
    
    plt.tight_layout()
    plt.show()

# Call the function
create_distance_box_plot(distance_results=distance_results, success_rates=success_rates, fig_size=(10, 8), fontsize=20)

### Distance Traveled / Success Rate Violin Plot

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

def adjacent_values(vals, q1, q3):
    """Calculate whisker positions for violin plot"""
    upper_adjacent_value = q3 + (q3 - q1) * 1.5
    upper_adjacent_value = np.clip(upper_adjacent_value, q3, vals[-1])

    lower_adjacent_value = q1 - (q3 - q1) * 1.5
    lower_adjacent_value = np.clip(lower_adjacent_value, vals[0], q1)
    return lower_adjacent_value, upper_adjacent_value

def create_distance_violin_plot(distance_results, success_rates, fig_size=(18, 6), fontsize=16):
    """
    Create a violin plot from distance_results with success rate bars.
   
    Args:
        distance_results: Dictionary from calculate_distance_traveled()
                         {controller: [distances]}
        success_rates: Dictionary with success rates
                      {controller: {'perfect': rate, 'faulty': rate, 'failed': rate}}
        fig_size: Figure size tuple
        fontsize: Font size for labels and text
    """
   
    # Convert distance_results to DataFrame for plotting
    plot_data = []
    controller_labels = {
        'p': 'Position',
        'a': 'Admittance',
        'dp': 'Dynamic Param'
    }
   
    for controller, distances in distance_results.items():
        for distance in distances:
            plot_data.append({
                'experiment_type': controller_labels.get(controller, controller),
                'distance_traveled': distance
            })
   
    distance_df = pd.DataFrame(plot_data)
    controller_types = list(distance_df['experiment_type'].unique())
   
    # Find the maximum distance across all data
    max_distance = distance_df['distance_traveled'].max()
   
    # Create the plot with dual y-axes
    fig, ax1 = plt.subplots(figsize=fig_size)
    ax2 = ax1.twinx()
   
    sns.set_theme(
        style='white',
        font="Times New Roman"
    )
   
    # Prepare data for violin plot
    violin_data = []
    for controller_type in controller_types:
        subset = distance_df[distance_df['experiment_type'] == controller_type]
        violin_data.append(subset['distance_traveled'].values)
    
    # Create violin plot on primary axis
    violin_positions = [x - 0.15 for x in range(len(controller_types))]
    parts = ax1.violinplot(
        violin_data, 
        positions=violin_positions,
        showmeans=False, 
        showmedians=False,
        showextrema=False,
        widths=0.3
    )
    
    # Style violin bodies with typical violin plot colors
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c']  # Default matplotlib colors
    for i, pc in enumerate(parts['bodies']):
        color = colors[i % len(colors)]
        pc.set_facecolor(color)
        pc.set_edgecolor('black')
        pc.set_alpha(0.7)
        pc.set_linewidth(1.5)
    
    # Calculate statistics for each violin individually
    quartile1 = []
    medians = []
    quartile3 = []
    means = []
    conf_intervals_95 = []
    
    for data in violin_data:
        data_sorted = np.sort(data)
        q1, median, q3 = np.percentile(data_sorted, [25, 50, 75])
        mean_val = np.mean(data_sorted)
        
        # Calculate 95% confidence interval for the median (notch equivalent)
        n = len(data_sorted)
        # Standard error for median
        se_median = 1.57 * (q3 - q1) / np.sqrt(n)  # Approximation for median SE
        ci_95_lower = median - 1.96 * se_median
        ci_95_upper = median + 1.96 * se_median
        
        quartile1.append(q1)
        medians.append(median)
        quartile3.append(q3)
        means.append(mean_val)
        conf_intervals_95.append([ci_95_lower, ci_95_upper])
    
    # Convert to arrays for easier handling
    quartile1 = np.array(quartile1)
    medians = np.array(medians)
    quartile3 = np.array(quartile3)
    means = np.array(means)
    conf_intervals_95 = np.array(conf_intervals_95)
    
    # Add quartile lines, 95% CI, medians, and means
    ax1.vlines(violin_positions, quartile1, quartile3, color='black', linestyle='-', lw=5, alpha=0.8)
    ax1.vlines(violin_positions, conf_intervals_95[:, 0], conf_intervals_95[:, 1], 
               color='red', linestyle='-', lw=2, alpha=0.9, label='95% CI')
    ax1.scatter(violin_positions, medians, marker='o', color='white', s=30, zorder=3, 
               edgecolor='black', linewidth=1)
    ax1.scatter(violin_positions, means, marker='D', color='red', s=25, zorder=3, 
               edgecolor='darkred', linewidth=1, alpha=0.8)
    
    # Add horizontal line at maximum distance
    ax1.axhline(y=max_distance, color='green', linestyle='--', linewidth=2, 
               label='Goal Distance', alpha=0.8)
    
    # Set up primary y-axis
    ax1.set_xlabel("", fontsize=fontsize, fontweight='semibold')
    ax1.set_ylabel("Distance (m)", fontsize=fontsize, fontweight='semibold')
    ax1.tick_params(axis='both', labelsize=fontsize)
    ax1.grid(True, alpha=0.3, linestyle='-', linewidth=0.5)
    ax1.set_xticks(range(len(controller_types)))
    ax1.set_xticklabels(controller_types)
   
    # Create stacked bar plot on secondary axis
    x_positions = range(len(controller_types))
    
    # Prepare data for stacked bars
    perfect_rates = []
    faulty_rates = []
    
    for i, controller_type in enumerate(controller_types):
        # Find original controller key
        controller_key = None
        for key, label in controller_labels.items():
            if label == controller_type:
                controller_key = key
                break
        
        if controller_key and controller_key in success_rates:
            rates = success_rates[controller_key]
            perfect_rates.append(rates['perfect'] * 100)
            faulty_rates.append(rates['faulty'] * 100)
        else:
            perfect_rates.append(0)
            faulty_rates.append(0)
    
    # Offset bars to be next to violins (slightly to the right)
    bar_width = 0.25
    bar_positions = [x + 0.15 for x in x_positions]
    
    # Create stacked bars with different colors and patterns
    bars1 = ax2.bar(bar_positions, perfect_rates, bar_width, 
                   label='Perfect', color='lightgreen', edgecolor='darkgreen', 
                   linewidth=1, alpha=0.8)
    bars2 = ax2.bar(bar_positions, faulty_rates, bar_width, 
                   bottom=perfect_rates, label='Faulty', color='orange', 
                   edgecolor='darkorange', linewidth=1, alpha=0.8, hatch='///')
    
    # Add text labels above bars
    for i, (perfect_rate, faulty_rate) in enumerate(zip(perfect_rates, faulty_rates)):
        x_pos = bar_positions[i]
        
        # Text above perfect bar (green section)
        if perfect_rate > 0:
            ax2.text(x_pos, perfect_rate + 0, f'{perfect_rate:.0f}%', 
                    ha='center', va='bottom', fontsize=fontsize-4, fontweight='bold')
        
        # Text above faulty bar (orange section) - shows combined perfect + faulty rate
        combined_rate = perfect_rate + faulty_rate
        if combined_rate > 0:
            ax2.text(x_pos, combined_rate + 0, f'{combined_rate:.0f}%', 
                    ha='center', va='bottom', fontsize=fontsize-4, fontweight='bold')
    
    # Set up secondary y-axis (aligned with primary axis)
    ax2.set_ylabel("Success Rate (%)", fontsize=fontsize, fontweight='semibold')
    ax2.tick_params(axis='y', labelsize=fontsize)
    
    # Align the two y-axes so 0 matches 0% and max_distance matches 100%
    ax1_min, ax1_max = ax1.get_ylim()
    ax1.set_ylim(0, max_distance * 1.1)  # Add some padding
    
    # Set the secondary axis to scale proportionally
    ax2.set_ylim(0, 100 * (max_distance * 1.1) / max_distance)
    
    # Add sample size annotations above each violin
    for i, controller_type in enumerate(controller_types):
        subset = distance_df[distance_df['experiment_type'] == controller_type]
        n = len(subset)
       
        # Get the maximum y-value for this violin
        violin_data_subset = subset['distance_traveled'].values
        upper_whisker = violin_data_subset.max()
       
        # Place text slightly above the upper whisker
        y_min, y_max = ax1.get_ylim()
        ax1.text(i - 0.15, upper_whisker + (y_max - y_min) * 0.02, f'n={n}', 
                 ha='center', va='bottom', fontsize=fontsize-2, fontweight='normal')
    
    # Create legend in top left
    # Combine legends from both axes
    lines1, labels1 = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left', 
              fontsize=fontsize-2, framealpha=0.9)
       
    # Save as png and eps
    plt.savefig("./figures/distance_violin_plot.png", dpi=300, bbox_inches='tight')
    plt.savefig("./figures/distance_violin_plot.eps", bbox_inches='tight')
    
    # Print violin plot statistics
    print("Violin plot statistics:")
    print(f"Maximum distance (Goal): {max_distance:.2f}m")
    for controller_type in controller_types:
        subset = distance_df[distance_df['experiment_type'] == controller_type]
        print(f"{controller_type}:")
        print(f"  Mean: {subset['distance_traveled'].mean():.2f}")
        print(f"  Median: {subset['distance_traveled'].median():.2f}")
        print(f"  Std Dev: {subset['distance_traveled'].std():.2f}")
    
    # Print success rate statistics
    print("\nSuccess rate statistics:")
    for controller_type in controller_types:
        # Find original controller key
        controller_key = None
        for key, label in controller_labels.items():
            if label == controller_type:
                controller_key = key
                break
        
        if controller_key and controller_key in success_rates:
            rates = success_rates[controller_key]
            print(f"{controller_type}:")
            print(f"  Perfect: {rates['perfect']*100:.1f}%")
            print(f"  Faulty: {rates['faulty']*100:.1f}%")
            print(f"  Failed: {rates['failed']*100:.1f}%")
    
    plt.tight_layout()
    plt.show()

# Call the function (replace your original function call with this)
create_distance_violin_plot(distance_results=distance_results, success_rates=success_rates, fig_size=(10, 8), fontsize=20)